In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, SubsetRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import torchvision.transforms as transforms

from torchvision.transforms.functional import to_pil_image

import warnings
warnings.filterwarnings(action='ignore') 

# 기본 설정

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
base_dir = '/content/gdrive/MyDrive/open (2)/'

In [ ]:


train_folder = glob.glob(base_dir + 'train/*')

In [ ]:
train_img_list = glob.glob(base_dir + 'train/*/*')

In [ ]:
aug_img_list = glob.glob(base_dir + 'aug/*/*')

In [ ]:
test_img_list = glob.glob(base_dir + 'test/*/*')

In [ ]:
train = pd.DataFrame(columns=['img_path', 'label'])
train['img_path'] = train_img_list
train['label'] = train['img_path'].apply(lambda x : str(x).split('/')[-2])

In [ ]:
aug = pd.DataFrame(columns=['img_path', 'label'])
aug['img_path'] = aug_img_list
aug['label'] = aug['img_path'].apply(lambda x : str(x).split('/')[-2])

In [ ]:
test = pd.DataFrame(columns=['img_path'])
test['img_path'] = test_img_list

In [ ]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])

In [ ]:
df = pd.concat([train,aug])

In [ ]:
df['label'] = df['label'].astype(int)

# Weighted Loss에 사용하기 위한 가중치

In [ ]:
class_counts = df['label'].value_counts()
class_weights = 1./class_counts
class_weights = class_weights/class_weights.min()
class_weights = class_weights.to_dict()
class_weights = {k: v for k, v in sorted(class_weights.items(), key=lambda item: item[0])}
class_weights = list(class_weights.values())
class_weights = torch.FloatTensor(class_weights).to(device)
print(class_weights)

tensor([36.2917,  2.8371,  6.0069,  4.1476, 31.1071, 12.0972,  8.7980,  6.7000,
        49.3019, 11.4605,  1.4754,  6.1338, 19.7955, 25.6176, 16.1296,  5.3765,
        47.5091, 12.8088,  1.0000], device='cuda:0')


# 커스텀 데이터셋, DataLoader 설정

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label, transforms=None):
        self.img_path_list = img_path_list
        self.label = label
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label is not None:
            label = self.label[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.HorizontalFlip(p=0.5),
                            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), 
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
train_set = CustomDataset(df['img_path'].values, df['label'].values, train_transform)
train_loader = DataLoader(train_set, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8,pin_memory=True)

In [ ]:
test_set = CustomDataset(test['img_path'].values, None, train_transform)
test_loader = DataLoader(test_set, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8,pin_memory=True)

# 모델 설정

In [ ]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torchinfo import summary

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=19):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b4(pretrained=True)            # efficientnet_b4를 base로 학습
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
model = BaseModel().to(device)
print(summary(model, input_size = (32,3,224,224)))

Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-cf984f9c.pth
100%|██████████| 47.2M/47.2M [00:04<00:00, 11.9MB/s]


Layer (type:depth-idx)                                       Output Shape              Param #
BaseModel                                                    [32, 19]                  --
├─EfficientNet: 1-1                                          [32, 1000]                --
│    └─Sequential: 2-1                                       [32, 1536, 7, 7]          --
│    │    └─Conv2dNormActivation: 3-1                        [32, 40, 112, 112]        1,160
│    │    └─Sequential: 3-2                                  [32, 24, 112, 112]        3,504
│    │    └─Sequential: 3-3                                  [32, 32, 56, 56]          48,118
│    │    └─Sequential: 3-4                                  [32, 48, 28, 28]          110,912
│    │    └─Sequential: 3-5                                  [32, 96, 14, 14]          638,700
│    │    └─Sequential: 3-6                                  [32, 136, 14, 14]         1,387,760
│    │    └─Sequential: 3-7                                  [32, 23

# Stratified K Fold 후 모델 실행

In [ ]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=CFG['SEED'])

In [ ]:
def evaluation(dataloader):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(dataloader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
        model.train()
    return _val_loss, _val_score

In [ ]:
# 모델
use_amp = True
model = BaseModel().to(device)
validation_loss = []
#criterion = nn.CrossEntropyLoss().to(device)
#criterion = FocalLoss()
criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,lr_lambda=lambda epoch: 0.95 ** epoch,last_epoch=-1,verbose=False)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_set,y=train_set.label)): # 위의 k-fold class 사용
    
    
    train_subsampler = SubsetRandomSampler(train_idx) # index 생성
    val_subsampler = SubsetRandomSampler(val_idx) # index 생성
    
    # sampler를 이용한 DataLoader 정의
    trainloader = torch.utils.data.DataLoader(train_set, batch_size=CFG['BATCH_SIZE'], sampler=train_subsampler,num_workers=8,pin_memory=True) # 해당하는 index 추출
    valloader = torch.utils.data.DataLoader(train_set, batch_size=CFG['BATCH_SIZE'], sampler=val_subsampler,num_workers=8,pin_memory=True)
    
    

    for epoch in range(CFG['EPOCHS']): # EPOCH 만큼 학습을 진행한다.

        for inputs, labels in tqdm(iter(trainloader)):
            inputs = inputs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device) 

            optimizer.zero_grad() # 최적화 초기화
            with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
              outputs = model(inputs) # 모델에 입력값 대입 후 예측값 산출
              loss = criterion(outputs, labels) # 손실 함수 계산

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            #loss.backward() # 손실 함수 기준으로 역전파 설정 
            #optimizer.step() # 역전파를 진행하고 가중치 업데이트
        scheduler.step()
    train_loss,train_score = evaluation(trainloader) # 학습 데이터의 f1 score
    val_loss,val_score = evaluation(valloader)
    print("k-fold", fold," Train Score: %.4f, Validation Score: %.4f" %(train_score, val_score)) 
    validation_loss.append(val_score)

validation_loss = np.array(validation_loss)
mean = np.mean(validation_loss)
std = np.std(validation_loss)
print("Validation Score: %.4f, ± %.4f" %(mean, std))


  0%|          | 0/232 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]

...

  0%|          | 0/232 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

k-fold 4  Train Score: 0.9999, Validation Score: 1.0000
Validation Score: 0.9893, ± 0.0150


In [ ]:
import pandas as pd
submit = pd.read_csv('/content/gdrive/MyDrive/open (2)/sample_submission.csv')

model.eval()

batch_index = 0

for i, (images) in tqdm(enumerate(test_loader)):
    images = images.to(device)
    outputs = model(images)
    batch_index = i * CFG['BATCH_SIZE']
    max_vals, max_indices = torch.max(outputs, 1)
    submit.iloc[batch_index:batch_index + CFG['BATCH_SIZE'], 1:] = max_indices.long().cpu().numpy()[:,np.newaxis]

0it [00:00, ?it/s]

In [ ]:
submit

,id,label
0,TEST_000,10
1,TEST_001,18
2,TEST_002,18
3,TEST_003,7
4,TEST_004,10
...,...,...
787,TEST_787,3
788,TEST_788,14
789,TEST_789,18
790,TEST_790,18


In [ ]:
submit.loc[submit['label'] == 0, 'label'] = '가구수정'
submit.loc[submit['label'] == 1, 'label'] = '걸레받이수정'
submit.loc[submit['label'] == 2, 'label'] = '곰팡이'
submit.loc[submit['label'] == 3, 'label'] = '꼬임'
submit.loc[submit['label'] == 4, 'label'] = '녹오염'
submit.loc[submit['label'] == 5, 'label'] = '들뜸'
submit.loc[submit['label'] == 6, 'label'] = '면불량'
submit.loc[submit['label'] == 7, 'label'] = '몰딩수정'
submit.loc[submit['label'] == 8, 'label'] = '반점'
submit.loc[submit['label'] == 9, 'label'] = '석고수정'
submit.loc[submit['label'] == 10, 'label'] = '오염'
submit.loc[submit['label'] == 11, 'label'] = '오타공'
submit.loc[submit['label'] == 12, 'label'] = '울음'
submit.loc[submit['label'] == 13, 'label'] = '이음부불량'
submit.loc[submit['label'] == 14, 'label'] = '창틀,문틀수정'
submit.loc[submit['label'] == 15, 'label'] = '터짐'
submit.loc[submit['label'] == 16, 'label'] = '틈새과다'
submit.loc[submit['label'] == 17, 'label'] = '피스'
submit.loc[submit['label'] == 18, 'label'] = '훼손'

In [ ]:
submit['label'].value_counts()

훼손         344
오염         226
면불량         47
꼬임          33
터짐          25
석고수정        14
오타공         14
곰팡이         14
몰딩수정        12
피스          10
걸레받이수정      10
울음           8
가구수정         8
들뜸           6
녹오염          6
이음부불량        5
창틀,문틀수정      5
틈새과다         4
반점           1
Name: label, dtype: int64

In [ ]:
submit.to_csv('baseline_submit (3).csv', index=False)